## import et preparation des 3 df de textes

In [1]:
from joblib import dump,load

In [26]:
df = load("df_translated.joblib")
df.description = df.description.astype("str")
df.description_fr = df.description_fr.astype("str")
df.description_en = df.description_en.astype("str")
# fusion de désignation et description
df["texte_non_traduit"] = df.apply(lambda row: row["designation"] + " " + row["description"] if ((row["description"]!='nan') and (row["designation"] != row["description"])) else row["designation"], axis=1)

df["texte_traduit_fr"] = df.apply(lambda row: row["designation_fr"] + " " + row["description_fr"] if ((row["description_fr"]!='nan') and (row["designation_fr"] != row["description_fr"])) else row["designation_fr"], axis=1)

df["texte_traduit_en"] = df.apply(lambda row: row["designation_en"] + " " + row["description_en"] if ((row["description_en"]!='nan') and (row["designation_en"] != row["description_en"])) else row["designation_en"], axis=1)

#retrait des colonnes initiales description et designation
to_drop=['description','designation','productid', 'imageid','langues détectées', 'designation_fr', 'designation_en',
       'description_fr', 'description_en']
df = df.drop(columns = to_drop)
df_texte_non_traduit = df.drop(columns =['texte_traduit_fr', 'texte_traduit_en'])
df_texte_traduit_fr = df.drop(columns =['texte_non_traduit', 'texte_traduit_en'])
df_texte_traduit_en = df.drop(columns =['texte_non_traduit', 'texte_traduit_fr'])

In [30]:
dump(df_texte_non_traduit,"df_texte_non_traduit.joblib")

['df_texte_non_traduit.joblib']

In [31]:
dump(df_texte_traduit_fr,"df_texte_traduit_fr.joblib")

['df_texte_traduit_fr.joblib']

In [ ]:
dump(df_texte_traduit_en,"df_texte_traduit_en.joblib")

In [36]:
df_texte_non_traduit = load("df_texte_non_traduit.joblib")
df_texte_traduit_fr = load("df_texte_traduit_fr.joblib")
df_texte_traduit_en = load("df_texte_traduit_en.joblib")

## Prepa des données FR avant DL

In [37]:
df_texte_traduit_fr

,prdtypecode,texte_traduit_fr
0,10,Olivia : Carnet personnalisé / 150 pages / Dot...
1,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,2705,La Guerre Des Tuques Luc a des id&eacute;es de...
...,...,...
84911,40,The Sims [ Import Anglais ]
84912,2583,Kit piscine acier NEVADA déco pierre Ø 3.50m x...
84913,2280,Journal Officiel De La Republique Francaise N°...
84914,1560,Table Basse Bois De Récupération Massif Base B...


In [39]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


stop_words = stopwords.words('french')

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df_texte_traduit_fr.texte_traduit_fr = df_texte_traduit_fr.texte_traduit_fr.apply(lambda x :preprocess_sentence(x))
df_texte_traduit_fr.head()


,prdtypecode,texte_traduit_fr
0,10,olivia carnet personnalise pages dot grid din ...
1,2280,journal arts art marche salon art asiatique pa...
2,50,grand stylet ergonomique bleu gamepad nintendo...
3,1280,peluche donald europe disneyland marionnette d...
4,2705,guerre tuques luc eacute grandeur . veut organ...


In [50]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit([  10, 2280,   50, 1280, 2705, 2522, 2582, 1560, 1281, 1920, 2403,
       1140, 2583, 1180, 1300, 2462, 1160, 2060,   40,   60, 1320, 1302,
       2220, 2905, 2585, 1940, 1301])

LabelEncoder()

In [60]:
from sklearn.model_selection import train_test_split

features_fr = df_texte_traduit_fr.texte_traduit_fr


target_fr = df_texte_traduit_fr.prdtypecode
target_fr_encoded = le.transform(target_fr)
X_text_train, X_text_test, y_train, y_test = train_test_split(features_fr, target_fr_encoded, test_size=0.2, random_state=123)

In [61]:
import tensorflow as tf
# Définition du tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
# Mettre à jour le dictionnaire du tokenizer
tokenizer.fit_on_texts(X_text_train)


In [62]:
# Définition des dictionnaires
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words

In [63]:
X_train = tokenizer.texts_to_sequences(X_text_train)

X_test = tokenizer.texts_to_sequences(X_text_test)

In [64]:
maxlen = 500
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')


In [65]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, RNN, GRUCell, Dropout
embedding_dim = 200

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(RNN(GRUCell(128), return_sequences=True))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(27, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 200)         2000000   
                                                                 
 rnn_2 (RNN)                 (None, None, 128)         126720    
                                                                 
 dropout_4 (Dropout)         (None, None, 128)         0         
                                                                 
 global_average_pooling1d_2  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                      

In [70]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, y_train,
    batch_size = 64,
    epochs=10,
    validation_data = [X_test, y_test])

Epoch 1/10
1062/1062 [==============================] - 1426s 1s/step - loss: 1.6919 - accuracy: 0.4919 - val_loss: 0.9530 - val_accuracy: 0.7159
Epoch 2/10
1062/1062 [==============================] - 1328s 1s/step - loss: 0.8218 - accuracy: 0.7549 - val_loss: 0.7917 - val_accuracy: 0.7665
Epoch 3/10
1062/1062 [==============================] - 2019s 2s/step - loss: 0.6323 - accuracy: 0.8101 - val_loss: 0.7578 - val_accuracy: 0.7759
Epoch 4/10
1062/1062 [==============================] - 1522s 1s/step - loss: 0.5169 - accuracy: 0.8416 - val_loss: 0.7622 - val_accuracy: 0.7800
Epoch 5/10
1062/1062 [==============================] - 1957s 2s/step - loss: 0.4345 - accuracy: 0.8641 - val_loss: 0.8098 - val_accuracy: 0.7816
Epoch 6/10
1062/1062 [==============================] - 1954s 2s/step - loss: 0.3695 - accuracy: 0.8824 - val_loss: 0.8671 - val_accuracy: 0.7776
Epoch 7/10
1062/1062 [==============================] - 2043s 2s/step - loss: 0.3143 - accuracy: 0.9004 - val_loss: 0.9525 -

In [71]:
model.evaluate(X_test, y_test, batch_size=128)

133/133 [==============================] - 59s 447ms/step - loss: 1.2239 - accuracy: 0.7730


[1.2239493131637573, 0.7729628086090088]

In [72]:
model.save("DL_text")

INFO:tensorflow:Assets written to: DL_text\assets


INFO:tensorflow:Assets written to: DL_text\assets
